## Load information 
### Gather solar radiation data from multiple files

In [1]:
import pandas as pd
import glob
import os

from shapely.geometry import Point
from helper import *

path = 'data/radiation/'
files = glob.glob(os.path.join(path, '*.csv'))

radiation_data = []
for f in files:
    # Read CSV skipping first two lines and using the third line as header
    df = pd.read_csv(f, skiprows=2)
    # Extract lat/lon from filename (e.g., xxxxxxx_41.88_-87.63_yyyy.csv)
    lat, lon = map(float, os.path.basename(f).split('_')[1:3])
    df['lat'] = lat
    df['lon'] = lon
    df['geometry'] = Point(lon, lat)
    radiation_data.append(df)

radiation_df = pd.concat(radiation_data, ignore_index=True)

Convert stadard dataframe into geopandas dataframe. This will still have multiple timestamps (year, month, day, hour) per (DHI, DNI, GHI) triplet.

In [2]:
import geopandas as gpd

radiation_gdf = gpd.GeoDataFrame(radiation_df, geometry='geometry', crs='EPSG:4326')
radiation_gdf

,Year,Month,Day,Hour,Minute,DHI,GHI,DNI,Solar Zenith Angle,lat,lon,geometry
0,2023,1,1,0,0,0,0,0,106.22,41.98,-87.73,POINT (-87.73 41.98)
1,2023,1,1,1,0,0,0,0,117.11,41.98,-87.73,POINT (-87.73 41.98)
2,2023,1,1,2,0,0,0,0,128.23,41.98,-87.73,POINT (-87.73 41.98)
3,2023,1,1,3,0,0,0,0,139.25,41.98,-87.73,POINT (-87.73 41.98)
4,2023,1,1,4,0,0,0,0,149.61,41.98,-87.73,POINT (-87.73 41.98)
...,...,...,...,...,...,...,...,...,...,...,...,...
6806515,2023,12,31,19,0,106,106,0,66.51,41.68,-87.71,POINT (-87.71 41.68)
6806516,2023,12,31,20,0,42,42,0,70.99,41.68,-87.71,POINT (-87.71 41.68)
6806517,2023,12,31,21,0,50,50,0,77.68,41.68,-87.71,POINT (-87.71 41.68)
6806518,2023,12,31,22,0,10,10,0,85.96,41.68,-87.71,POINT (-87.71 41.68)


In [3]:
print(radiation_gdf.crs)

EPSG:4326


To have a single geopandas data frame with a single triplet per coordinate which will be the year average.

In [4]:
# not removing zeros because those were measurements, not nan
sum_gdf = radiation_gdf.groupby(['lat', 'lon']).agg({
    "GHI": ["mean", "sum"],
    "DNI": ["mean", "sum"],
    "DHI": ["mean", "sum"]
}) 

# Flatten the MultiIndex columns
sum_gdf.columns = ['_'.join(col).strip() for col in sum_gdf.columns.values]

# Reset index to make it easier to join back later
sum_gdf = sum_gdf.reset_index()

sum_gdf['geometry'] = sum_gdf.apply(lambda row: Point(row['lon'], row['lat']), axis=1)
sum_gdf['GHI/DNI'] = sum_gdf['GHI_mean']/sum_gdf['DNI_mean']
sum_gdf = gpd.GeoDataFrame(sum_gdf, geometry='geometry', crs='EPSG:4326') 
sum_gdf = sum_gdf.to_crs('EPSG:3435')  # Chicago's local CRS
sum_gdf

,lat,lon,GHI_mean,GHI_sum,DNI_mean,DNI_sum,DHI_mean,DHI_sum,geometry,GHI/DNI
0,41.64,-87.97,165.969064,1453889,182.585274,1599447,62.619178,548544,POINT (1083562.578 1811618.167),0.908995
1,41.64,-87.95,165.943265,1453663,182.646918,1599987,62.639384,548721,POINT (1089029.391 1811641.853),0.908547
2,41.64,-87.93,165.955936,1453774,182.663584,1600133,62.632192,548658,POINT (1094496.205 1811666.806),0.908533
3,41.64,-87.91,166.399315,1457658,185.094749,1621430,61.825799,541594,POINT (1099963.021 1811693.028),0.898995
4,41.64,-87.89,166.332648,1457074,185.556164,1625472,61.631849,539895,POINT (1105429.838 1811720.517),0.896401
...,...,...,...,...,...,...,...,...,...,...
772,42.04,-87.33,158.380251,1387411,160.960616,1410015,64.334817,563573,POINT (1256798.315 1958764.788),0.983969
773,42.04,-87.31,158.618721,1389500,160.726256,1407962,64.344977,563662,POINT (1262231.265 1958829.138),0.986887
774,42.04,-87.29,158.655137,1389819,160.997489,1410338,64.251484,562843,POINT (1267664.219 1958894.757),0.985451
775,42.04,-87.27,158.976712,1392636,161.735845,1416806,64.047603,561057,POINT (1273097.176 1958961.648),0.982941


### Load building footprints

In [5]:
buildings_gdf = gpd.read_file('data/footprints/Buildings.shp')

In [6]:
print('buildings gdf crs: ', buildings_gdf.crs)
print('buildings gdf shape: ', buildings_gdf.shape)

buildings gdf crs:  EPSG:3435
buildings gdf shape:  (820154, 43)


## Match radiation points to building footprints

In [7]:
joined_gdf = buildings_gdf.sjoin_nearest(sum_gdf, how='left', distance_col='distance_meters')

In [8]:
# check if the join worked properly and see how far off are the interpolations
(joined_gdf.sort_values(by='distance_meters', ascending=True)).tail(25)

,BLDG_ID,CDB_CITY_I,BLDG_STATU,F_ADD1,T_ADD1,PRE_DIR1,ST_NAME1,ST_TYPE1,UNIT_NAME,NON_STANDA,...,lat,lon,GHI_mean,GHI_sum,DNI_mean,DNI_sum,DHI_mean,DHI_sum,GHI/DNI,distance_meters
58392,617574,None,ACTIVE,7923,7923,S,LOWE,AVE,None,None,...,41.74,-87.65,164.101941,1437533,176.275228,1544171,63.545548,556659,0.930942,4510.665344
333703,564558,None,ACTIVE,6802,6802,S,BELL,AVE,None,None,...,41.76,-87.67,165.561301,1450317,180.846119,1584212,63.236986,553956,0.915482,4511.050973
695387,166425,None,ACTIVE,0,0,None,None,None,None,None,...,41.94,-87.67,162.751712,1425705,172.719977,1513027,64.082192,561360,0.942287,4511.220292
525360,308931,None,ACTIVE,1633,1633,N,SAYRE,AVE,None,None,...,41.92,-87.79,164.720320,1442950,179.076941,1568714,63.232991,553921,0.919830,4511.271270
251595,248950,None,ACTIVE,0,0,None,None,None,None,None,...,41.92,-87.73,164.314954,1439399,179.137671,1569246,62.620205,548553,0.917255,4512.118397
666282,168143,None,ACTIVE,0,0,None,None,None,None,None,...,41.96,-87.77,164.217694,1438547,179.457534,1572048,62.583790,548234,0.915078,4512.291953
448692,771680,None,ACTIVE,11200,11218,S,TORRENCE,AVE,OOM,None,...,41.70,-87.57,163.784018,1434748,175.179795,1534575,63.544635,556651,0.934948,4512.398303
469549,564225,None,ACTIVE,6811,6811,S,BISHOP,ST,None,None,...,41.78,-87.67,165.313470,1448146,180.029680,1577060,63.085388,552628,0.918257,4513.476365
496491,500270,None,ACTIVE,5605,5605,S,NEVA,AVE,None,None,...,41.80,-87.79,165.449658,1449339,179.960959,1576458,63.711872,558116,0.919364,4513.921598
138285,682240,None,ACTIVE,0,0,None,None,None,None,None,...,41.74,-87.57,161.560731,1415272,171.170548,1499454,63.297831,554489,0.943858,4514.030727


Compute surface area in $m^2$ from geometry

In [9]:
# Convert to projected CRS (e.g., for Chicago, use EPSG:26971 or EPSG:3857)
joined_gdf = joined_gdf.to_crs(epsg=3857)  # NAD83 / Illinois East (ft); or use 3857 for meters

# Compute surface area in m²
joined_gdf["surface_area"] = joined_gdf.geometry.area

# Convert back to lat/lon
joined_gdf = joined_gdf.to_crs(epsg=3435)

## Compute solar potential

To first order, 

$kWh = GHI × area × efficiency / 100$

so we can already compute the annual solar energy output at each building given that

- We have the column GHI_sum (Wh/m²/year)
- We have the column SHAPE_AREA which is the roof surface area in m²
- We can assume that the system efficiency is ~15%

In [10]:
# trim data to avoid slowness during visualizations
gdf = joined_gdf.copy()
gdf = gdf[['GHI_sum', 'GHI/DNI', 'lon', 'lat', 'BLDG_ID', 'surface_area', 'geometry']]
gdf.columns = [col.lower() for col in gdf.columns]

In [11]:
# compute orientation of the longest edge as a proxy for roof orientation (azimuth)
gdf["orientation_angle"] = gdf.geometry.apply(get_orientation)

# classify orientation (cardinal direction)
gdf["orientation"] = gdf["orientation_angle"].apply(azimuth_to_orientation)

In [12]:
print(gdf.columns.tolist())

['ghi_sum', 'ghi/dni', 'lon', 'lat', 'bldg_id', 'surface_area', 'geometry', 'orientation_angle', 'orientation']


In [13]:
# estimate kwh per year
gdf["kwh_estimate"] = gdf.apply(
    lambda row: get_kwh(row["ghi_sum"], row["surface_area"]),
    axis=1
)

In [14]:
# save geodataframe
gdf.to_file('data/solar_summary.geojson', driver='GeoJSON')

Sort buildings from highest to lowest potential energy output and save the top 100.

In [15]:
col_pop = 'kwh_estimate'
gdf.insert(0, col_pop, gdf.pop(col_pop))

gdf_sorted = gdf.sort_values(by="kwh_estimate", ascending=False)

# save to geojson
top_k = gdf_sorted.head(100)
top_k.to_file("top_100_buildings.geojson", driver="GeoJSON")
top_k.drop(columns="geometry").to_csv("top_100_buildings.csv", index=False)
top_k.tail()

,kwh_estimate,ghi_sum,ghi/dni,lon,lat,bldg_id,surface_area,geometry,orientation_angle,orientation
433699,1.010428e+07,1399715,0.935650,-87.63,41.88,881847,48125.418873,"POLYGON ((1177644.643 1899875.539, 1177654.143...",91.673504,East
437372,1.003923e+07,1444531,0.927468,-87.63,41.78,559877,46332.138907,"POLYGON ((1175910.643 1859293.039, 1175909.643...",271.782790,West
304006,1.003525e+07,1422919,0.948122,-87.65,41.82,432266,47017.180762,"POLYGON ((1171410.147 1875359.567, 1171398.645...",271.569983,West
730874,1.002471e+07,1398476,0.940086,-87.65,41.90,321960,47788.717071,"POLYGON ((1170281.643 1907175.539, 1170074.143...",302.631489,Northwest
509056,1.001418e+07,1448184,0.913834,-87.81,41.98,46929,46099.955435,"POLYGON ((1125329.643 1938805.039, 1125341.643...",26.068279,Northeast
